In [1]:
#import libraries
import torch
import torch.nn as nn
import numpy as np


In [2]:
#import dataset
import seaborn as sns
iris = sns.load_dataset('iris')

#convert from pandas dataframe to tensor
data = torch.tensor( iris[iris.columns[0:4]].values ).float()

# transform species to number
labels = torch.zeros(len(data), dtype=torch.long)
# labels[iris.species=='setosa'] = 0 # don't need!
labels[iris.species=='versicolor'] = 1
labels[iris.species=='virginica'] = 2

# Separate  data into train and test set

In [3]:
# no devset here
#how many training examples
propTraining = .8 # 80% data in the training set and 20 will be in test set
nTraining = int(len(labels)*propTraining)

#initialize a boolean vector to select data and labels
traintestBool = np.zeros(len(labels),dtype=bool)

#is this correct way to select samples?
#traintestBool[range(nTraining)] = True

#this is better
items2use4train = np.random.choice(range(len(labels)),nTraining,replace=False)
traintestBool[items2use4train]=True

traintestBool


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True, False,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True, False,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True, False,  True, False,  True,  True,  True,  True,  True,
        True, False,  True,  True, False,  True,  True, False,  True,
        True, False,  True, False,  True,  True,  True,  True,  True,
       False, False,  True,  True,  True,  True, False, False,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False,  True, False,
       False,  True,  True, False, False, False,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True, False,

In [4]:
# test whether its balanced
print("Average of full data:")
print( torch.mean(labels.float()) ) # =1 by definition
print(' ')

print('Average of training data:')
print( torch.mean(labels[traintestBool].float()) ) # should be 1...
print(' ')


print('Average of test data:')
print( torch.mean(labels[~traintestBool].float()) ) # should also be 1...

Average of full data:
tensor(1.)
 
Average of training data:
tensor(0.9667)
 
Average of test data:
tensor(1.1333)


In [6]:
# create the ANN model

#model architecture
ANNiris = nn.Sequential(
    nn.Linear(4,64),
    nn.ReLU(),         # activation unit
    nn.Linear(64,64),  # hidden layer
    nn.ReLU(),         # activation unit
    nn.Linear(64,3),   # output units
      )

#loss function
lossfun = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(ANNiris.parameters(),lr=.01)


In [7]:
#entire dataset
print(data.shape)

#training set
print(data[traintestBool,:].shape)

# test set
print( data[~traintestBool,:].shape )

torch.Size([150, 4])
torch.Size([120, 4])
torch.Size([30, 4])


# Train and test the model

In [8]:
# train the model
numepochs = 1000

#initialize losses
losses = torch.zeros(numepochs)
ongoingAcc = []

#loop over epochs
for epochi in range(numepochs):
    # forward pass
    yHat = ANNiris(data[traintestBool,:])

    # compute accuracy (note: denser than previous code!)
    ongoingAcc.append( 100*torch.mean(

        (torch.argmax(yHat,axis=1) == labels[traintestBool]).float()) )

    # compute loss
    loss = lossfun(yHat,labels[traintestBool])
    losses[epochi] = loss

    # backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [9]:
#compute train and test accuracies
#final forward pass using training data.

predictions = ANNiris(data[traintestBool,:])
trainacc = 100*torch.mean((torch.argmax(predictions,axis=1) == labels[traintestBool]).float())


# final forward pass USING TEST DATA!
predictions = ANNiris(data[~traintestBool,:])
testacc = 100*torch.mean((torch.argmax(predictions,axis=1) == labels[~traintestBool]).float())


In [10]:
# report accuracies

print('Final TRAIN accuracy: %g%%' %trainacc)
print('Final TEST accuracy:  %g%%' %testacc)


Final TRAIN accuracy: 98.3333%
Final TEST accuracy:  96.6667%
